# Getting Started with Llama2



# Setup

## Quick Start

Below I am just copy pasting the quick start setup from the [official repo](https://github.com/facebookresearch/llama) just to help out.

You can follow the steps below to quickly get up and running with Llama 2 models. These steps will let you run quick inference locally. For more examples, see the Llama 2 recipes repository.

- In a conda env with PyTorch / CUDA available clone and download this repository.

- In the top level directory run:

```pip install -e .```

- Visit the [Meta website and register](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) to download the model/s.

- Once registered, you will get an email with a URL to download the models. You will need this URL when you run the download.sh script.

- Once you get the email, navigate to your downloaded llama repository and run the download.sh script.

- Make sure to grant execution permissions to the download.sh script

- During this process, you will be prompted to enter the URL from the email.

- Do not use the “Copy Link” option but rather make sure to manually copy the link from the email.

- Once the model/s you want have been downloaded, you can run the model locally using the command below:

```
torchrun --nproc_per_node 1 example_chat_completion.py \
    --ckpt_dir llama-2-7b-chat/ \
    --tokenizer_path tokenizer.model \
    --max_seq_len 512 --max_batch_size 6
```

### Note

- Replace llama-2-7b-chat/ with the path to your checkpoint directory and tokenizer.model with the path to your tokenizer model.
- The –nproc_per_node should be set to the MP value for the model you are using.
- Adjust the max_seq_len and max_batch_size parameters as needed.
- This example runs the example_chat_completion.py found in this repository but you can change that to a different .py file.

> All of this info can be found in the Meta's [official repo](https://github.com/facebookresearch/llama)!

## You can also access the weights through [Hugging Face](https://huggingface.co/meta-llama):

- You must first request a download from the Meta website using the same email address as your Hugging Face account.
- After that you can request access to any of the models on Hugging Face and within 1-2 days your account will be granted access to all versions.

[Go through this recipes setup from the official repository from Meta](https://github.com/facebookresearch/llama)

Ok great! Once you've done all of that you should have a folder with your chosen model, in my case I have 2 quantized 7b-chat models:

In [3]:
!ls models/

llama-2-7b-chat.Q4_K_M.gguf llama-2-7b-chat.Q5_K_M.gguf


Now, lets look at how to run inference with Llama2 using a Python-friendly approach!

For that we'll be looking at a few awesome options for running inference with llama2 models including:

- [llama-cpp](https://github.com/ggerganov/llama.cpp) library
- [`llama-cpp-python`](https://github.com/abetlen/llama-cpp-python) library, which presents Python bindings for the [llama-cpp](https://github.com/ggerganov/llama.cpp) library.
- Running [llama2 with Hugging Face](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF)
- Setup with [Llama-cpp-python + Langchain](https://python.langchain.com/docs/integrations/llms/llamacpp)
- And finally a setup using [Ollama](https://ollama.ai/) and [Ollama+Langchain](https://python.langchain.com/docs/integrations/llms/ollama)
- Also using [LM-Studio](https://lmstudio.ai/).

Why so many options? Because its important to know that there are many ways to run these models and I think its nice to present a landscape of implementational options :) .

# [Llama-cpp (original)](https://github.com/ggerganov/llama.cpp)

```
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp

On Linux/Mac
- make

On Windows:
- Download the latest fortran version of w64devkit.
- Extract w64devkit on your pc.
- Run w64devkit.exe.
- Use the cd command to reach the llama.cpp folder.
- From here you can run:
- make
```

Then all you have to do is:

```
# obtain the original LLaMA model weights and place them in ./models
ls ./models
65B 30B 13B 7B tokenizer_checklist.chk tokenizer.model
  # [Optional] for models using BPE tokenizers
  ls ./models
  65B 30B 13B 7B vocab.json

# install Python dependencies
python3 -m pip install -r requirements.txt

# convert the 7B model to ggml FP16 format
python3 convert.py models/7B/

  # [Optional] for models using BPE tokenizers
  python convert.py models/7B/ --vocabtype bpe

# quantize the model to 4-bits (using q4_0 method)
./quantize ./models/7B/ggml-model-f16.gguf ./models/7B/ggml-model-q4_0.gguf q4_0

# update the gguf filetype to current if older version is unsupported by another application
./quantize ./models/7B/ggml-model-q4_0.gguf ./models/7B/ggml-model-q4_0-v2.gguf COPY


# run the inference
./main -m ./models/7B/ggml-model-q4_0.gguf -n 128
```



# Llama-cpp-Python

- [source](https://abetlen.github.io/llama-cpp-python/)

Let's start by installing the package:

In [3]:
# !pip install llama-cpp-python

In [4]:
# source for the weights: https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

Now we can load the `Llama` class.

In [1]:
from llama_cpp import Llama

Now we just instantiate the llama object giving the path to the downloaded weights for our model of choice.

In [2]:
llm = Llama(model_path="./models/llama-2-7b-chat.Q4_K_M.gguf", n_gpu_layers=1, n_ctx=4096,verbose=False)

Now, running predictions for [`Text Completion`](https://github.com/abetlen/llama-cpp-python#:~:text=Below%20is%20a,28%2C%0A%20%20%20%20%22total_tokens%22%3A%2042%0A%20%20%7D%0A%7D) is a breeze.

In [3]:
output = llm("What is a large language model?.")
output

{'id': 'cmpl-b359b746-9e97-40b3-a6c1-7f299014cdab',
 'object': 'text_completion',
 'created': 1712418188,
 'model': './models/llama-2-7b-chat.Q4_K_M.gguf',
 'choices': [{'text': ' A large language model is a deep learning model that is trained on a vast amount',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 8, 'completion_tokens': 16, 'total_tokens': 24}}

This schema is very similar to the one we are all used to with the ChatGPT API.

In [4]:
print(output["choices"][0]["text"])

 A large language model is a deep learning model that is trained on a vast amount


Notice that here we are getting just questions, because this is the "base model" (which for questions will just output more questions).

See Andrej Karpathy explain that in [this part of his youtube video introducing LLMs](https://youtu.be/zjkBMFhNj_g?t=1213)

For the actually useful model we specify the `chat_format` parameter in the `Llama` class. 

Now, when we ask the same question:

In [5]:
llm = Llama(model_path="./models/llama-2-7b-chat.Q4_K_M.gguf", chat_format="llama-2", verbose=False)

prompt = "What is a large language model?."
response = llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "You are a helpful assistant."},
          {
              "role": "user",
              "content": prompt
          }
      ]
)
print(response)
response["choices"][0]["message"]["content"]

{'id': 'chatcmpl-3d711b28-21c4-4f11-8ef5-9e4167b527c2', 'object': 'chat.completion', 'created': 1712418276, 'model': './models/llama-2-7b-chat.Q4_K_M.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '  A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. The goal of an LLM is to learn the patterns and structures of a language, such as grammar, syntax, and semantics, in order to produce text that is similar to human language.\nLLMs are typically trained on vast amounts of text data, such as books, articles, or websites, and use a combination of machine learning algorithms and neural networks to learn the patterns and relationships between words and phrases in the language. Once trained, an LLM can be used for a variety of natural language processing tasks, such as:\n1. Language translation: An LLM can be trained to transla

'  A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. The goal of an LLM is to learn the patterns and structures of a language, such as grammar, syntax, and semantics, in order to produce text that is similar to human language.\nLLMs are typically trained on vast amounts of text data, such as books, articles, or websites, and use a combination of machine learning algorithms and neural networks to learn the patterns and relationships between words and phrases in the language. Once trained, an LLM can be used for a variety of natural language processing tasks, such as:\n1. Language translation: An LLM can be trained to translate text from one language to another.\n2. Text summarization: An LLM can be used to summarize long pieces of text into shorter, more digestible versions.\n3. Sentiment analysis: An LLM can be trained to analyze the sentiment o

We get an actual response from the model.

# [Llama2 inference with Hugging Face](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF)


First we would download the model (in case you haven't before).

In [6]:
# Uncomment below if you are on google colab and don't have the llama2 7b model downloaded
# !pip install huggingface-cli
# !huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF llama-2-7b-chat.Q4_K_M.gguf --local-dir ./models --local-dir-use-symlinks False

Now we'll need to install the ctransformer library, check [here](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF#:~:text=Python%20using%20ctransformers-,First%20install%20the%20package,0.2.24%20--no-binary%20ctransformers,-Simple%20example%20code) for the option that makes sense for your machine.

In [8]:
# source: https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

# # Base ctransformers with no GPU acceleration
# pip install ctransformers>=0.2.24
# # Or with CUDA GPU acceleration
# pip install ctransformers[cuda]>=0.2.24
# # Or with ROCm GPU acceleration
# CT_HIPBLAS=1 pip install ctransformers>=0.2.24 --no-binary ctransformers


# # Or with Metal GPU acceleration for macOS systems ---- I am in a MacOS machine so I'll use this option!
# !CT_METAL=1 pip install ctransformers --no-binary ctransformers
# This is the only package that is not included in the requirements because it will be installed in a different way depending on the system.

Example to load and run a GGUF model:

In [9]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
#llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF", model_file="llama-2-7b-chat.Q4_K_M.gguf", model_type="llama", gpu_layers=50)
llm = AutoModelForCausalLM.from_pretrained(model_path_or_repo_id="./models/llama-2-7b-chat.Q4_K_M.gguf", model_type="llama")
print(llm("What is a large language model?."))

objc[58244]: Class GGMLMetalClass is implemented in both /Users/greatmaster/miniconda3/envs/oreilly-llama2/lib/python3.10/site-packages/llama_cpp/libllama.dylib (0x11913c250) and /Users/greatmaster/miniconda3/envs/oreilly-llama2/lib/python3.10/site-packages/ctransformers/lib/local/libctransformers.dylib (0x12cc741d0). One of the two will be used. Which one is undefined.



 everyone has heard of chatbots, but what exactly are they? In this article, we will explore the concept of chatbots, how they work, and their potential applications. A chatbot is a computer program that mimics human conversation, either through text or voice interactions. The term "chatbot" was coined in 1987, but the technology has been around for much longer. In this article, we will explore the history of chatbots and how they have evolved over time.
A large language model is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. These models have become increasingly popular in recent years due to their ability to generate text that is often indistinguishable from human-written text. In this article, we will explore the inner workings of a large language model, how it is trained, and some of its potential applications.
The term "large language model" can be somewhat misleading

In [10]:
print(llm("List 10 essential stretches a person should do to live longer, healthier and stronger."))

 Here are ten of the best stretches that you can do at home or in the office:
Stretching is an important part of any fitness routine, as it helps improve flexibility, reduce muscle tension, and enhance overall well-being. Incorporating stretching into your daily routine can help you live longer, healthier, and stronger. Here are ten essential stretches that everyone should do to achieve these benefits:
1. Neck Stretch: Slowly tilt your head to the side, bringing your ear towards your shoulder. Hold for 30 seconds and repeat on the other side. This stretch helps relieve tension in the neck and improves posture.
2. Shoulder Rolls: Roll your shoulders forward and backward in a circular motion. Repeat for 10-15 repetitions to improve mobility in the shoulders.
3. Chest Stretch: Place your hands on a wall or door frame and lean forward, stretching your chest. Hold for 30 seconds and repeat several times to improve flexibility in the upper body.
4. Arm Circles: Hold your arms straight out to

You can also check out this demo available in the hugging face website for interacting with the Llama2-70B-Chat model:
- https://huggingface.co/spaces/ysharma/Explore_llamav2_with_TGI

# Setup with LangChain

[source](https://python.langchain.com/docs/integrations/llms/llamacpp)

Let's install the langchain package

In [12]:
# if you are on google colab uncomment this and install langchain
# !pip install langchain

In [13]:
# source: https://python.langchain.com/docs/integrations/llms/llamacpp

# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [14]:
!CT_METAL=1
n_gpu_layers = 50  # Change this value based on your model and your GPU VRAM pool.
n_batch = 4096  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="./models/llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32       

In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What is the secret to becoming a great pragmatic programmer?"
llm_chain.invoke(question)

 ☕


Llama.generate: prefix-match hit



Step 1: Learn Pragmatic Programming Basics 📚
To become a skilled pragmatic programmer, you must first understand the fundamentals of the field. Start by reading books like "Pragmatic Programmer" by Andrew Hunt and David Thomas or "Clean Code" by Robert C. Martin. These resources will give you a solid foundation in pragmatic programming principles and practices. 📚
Step 2: Practice Pragmatism in Your Daily Work 💻
The best way to become a great pragmatic programmer is to practice pragmatism in your daily work. This means focusing on delivering working software, writing clean and maintainable code, and constantly improving your craft. As you gain experience, you'll find yourself naturally incorporating pragmatic principles into your workflow. 💻
Step 3: Learn from Others 🤝
Seek out other pragmatic programmers and learn from their experiences. Attend conferences, join online communities, and participate in code reviews to share knowledge and best practices. By collaborating with


llama_print_timings:        load time =    2910.85 ms
llama_print_timings:      sample time =      32.05 ms /   256 runs   (    0.13 ms per token,  7986.77 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    8123.52 ms /   256 runs   (   31.73 ms per token,    31.51 tokens per second)
llama_print_timings:       total time =    8816.87 ms /   257 tokens


{'question': 'What is the secret to becoming a great pragmatic programmer?',
 'text': ' ☕\n\nStep 1: Learn Pragmatic Programming Basics 📚\nTo become a skilled pragmatic programmer, you must first understand the fundamentals of the field. Start by reading books like "Pragmatic Programmer" by Andrew Hunt and David Thomas or "Clean Code" by Robert C. Martin. These resources will give you a solid foundation in pragmatic programming principles and practices. 📚\nStep 2: Practice Pragmatism in Your Daily Work 💻\nThe best way to become a great pragmatic programmer is to practice pragmatism in your daily work. This means focusing on delivering working software, writing clean and maintainable code, and constantly improving your craft. As you gain experience, you\'ll find yourself naturally incorporating pragmatic principles into your workflow. 💻\nStep 3: Learn from Others 🤝\nSeek out other pragmatic programmers and learn from their experiences. Attend conferences, join online communities, and pa

# Setup with OLLAMA and Ollama + Langchain

You can download the executable from the Ollama website and use it out of the box:
- https://ollama.ai/

In [19]:
# !pip install ollama
# Uncomment below if you haven't run this yet on your local machine or google colab
# !ollama pull llama2:latest

In [ ]:
import ollama

response = ollama.chat(model='llama2', 
                       messages=[
  {
    'role': 'user',
    'content': 'What is a large language model?',
  },
])

print(response['message']['content'])

## Set up Ollama with Langchain

You can also use `langchain` to use Ollama like shown below.

In [21]:
from langchain.chat_models import ChatOllama

ollama = ChatOllama(model="llama2")

ollama.predict("What is a large language model?")

' A large language model is a type of artificial intelligence (AI) model that is designed to understand and generate human-like text. It\'s called "large" because it typically involves training on a vast amount of data, which can range from millions to billions of words. These models use deep learning techniques, specifically a variant of neural networks called recurrent neural networks or transformers, to learn the statistical patterns in the data and generate coherent and contextually relevant responses. They are capable of performing various natural language processing tasks such as text completion, translation, summarization, and question answering. These models have achieved impressive results in understanding and generating human-like text and have found applications in various fields including customer service, content generation, and language translation.'

# Set up with LM-Studio

LM studio is like an improved version of gpt4all with additional features like automatic evaluation of your machine specs to know whether or not it can run a certain model:

![](./assets-resources/lm-studio-machine-specs-analysis.png)

ANother great feature of LM studio is the local inference server so you can host your own model:

![](./assets-resources/lm-studio-local-inf-server.png)

Super easy to download open source models:

![](./assets-resources/lm-studio-download.png)

After downloading it, I can select it for chat easily:

![](./assets-resources/lm-studio-select-to-chat.png)

And now you can just chat with that model:

![](./assets-resources/lm-studio-chat.png)

You can easily host and run inference on your own models:

In [1]:
# Load a model, start the server, and run this example in your terminal
# Choose between streaming and non-streaming mode by setting the "stream" field

!curl http://localhost:1234/v1/chat/completions -H "Content-Type: application/json" -d \
  '{"messages": [{"role": "system", "content": "Always answer in rhymes."},\
    {"role": "user", "content": "Introduce yourself."}\
  ], \
  "temperature": 0.7, \ 
  "max_tokens": -1,\
  "stream": false\
}'

{
  "id": "chatcmpl-4rxck8soi2xctzrc3zkauj",
  "object": "chat.completion",
  "created": 1707168904,
  "model": "/Users/greatmaster/.cache/lm-studio/models/TheBloke/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q2_K.gguf",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I'm a lively soul, with a heart that's whole."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 16,
    "total_tokens": 32
  }
}

Below are the results of running the inference above:

![](./assets-resources/lm-studio-local-server.png)

# Some Extra notes on running Llama on your phone

[MLC LLM](https://github.com/mlc-ai/mlc-llm) is an open-source project that makes it possible to run language models locally on a variety of devices and platforms, including iOS and Android.

For iPhone users, there’s an MLC chat app on the App Store. MLC now has support for the 7B, 13B, and 70B versions of Llama 2, but it’s still in beta and not yet on the Apple Store version, so you’ll need to install TestFlight to try it out. Check out out the [instructions for installing the beta version here](https://llm.mlc.ai/docs/index.html#getting-started).

# Final Note

The setup we're going to use will depend on the type of application we are trying to build.

But the overall rule for this course will be to use `llama-cpp-python` combined with langchain for ease of use for the usecases we are interested in.

# Resources

- https://github.com/abetlen/llama-cpp-python
- https://github.com/ggerganov/llama.cpp
- https://ai.meta.com/llama/
- https://ollama.ai/download
- https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF
- https://huggingface.co/spaces/ysharma/Explore_llamav2_with_TGI
- https://huggingface.co/blog/llama2
- https://replicate.com/blog/run-llama-locally
- https://ollama.ai/
- https://arxiv.org/pdf/2307.09288.pdf 
- https://ai.meta.com/llama/#resources
- https://www.philschmid.de/llama-2
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard